In [1]:
import os
import numpy as np

In [2]:
from exp_factory_functions import *

In [156]:
## Important Paths
# Cohort 1
c1_expOffboardingPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_1/battery_offboarding/'
c1_expOnboardingPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_1/battery_onboarding/'

c1_qualtricsOnboardingPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_1/run1_qualtrics_onboarding.csv'
c1_qualtricsOffboardingPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_1/run1_qualtricsOffboarding.csv'

c1_subjectsPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_1/run1_subjects.csv'

c1_outputDirectory = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_2_processed_data/run_1'

# Cohort 2
c2_expOffboardingPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_2/battery/run2_battery_offboarding/'
c2_expOnboardingPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_2/battery/run2_battery_onboarding/'

c2_qualtricsOnboardingPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_2/run2_on_ibGap_part1.csv'
c2_qualtricsOffboardingPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_2/run2_qualtricsOffboarding.csv'

c2_subjectsPath = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_2/run2_subjects.csv'

c2_outputDirectory = '/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_2_processed_data/run_2'


In [92]:
## Load cohort participants
c1_subjects = pd.read_csv(c1_subjectsPath)
c1_subjects = c1_subjects.rename(columns={'Student Number': 'student_id'})

c2_subjects = pd.read_csv(c2_subjectsPath)
c2_subjects = c2_subjects.rename(columns={'Student Number': 'student_id'})

In [176]:
## Onboarding || Offboarding + Cohort Switch

dataSource = 'c2_off'

match dataSource:
    case "c1_on":
        subjects = c1_subjects
        qualtrics_path = c1_qualtricsOnboardingPath
        expfactoryPath = c1_expOnboardingPath
        outputDirectory = c1_outputDirectory + "/battery_cohort1_onB.csv"

    case "c1_off":
        subjects = c1_subjects
        qualtrics_path = c1_qualtricsOffboardingPath
        expfactoryPath = c1_expOffboardingPath
        outputDirectory = c1_outputDirectory + "/battery_cohort1_offB.csv"

    case "c2_on":
        subjects = c2_subjects
        qualtrics_path = c2_qualtricsOnboardingPath
        expfactoryPath = c2_expOnboardingPath
        outputDirectory = c2_outputDirectory + "/battery_cohort2_onB.csv"
    
    case "c2_off":
        subjects = c2_subjects
        qualtrics_path = c2_qualtricsOffboardingPath
        expfactoryPath = c2_expOffboardingPath
        outputDirectory = c2_outputDirectory + "/battery_cohort2_offB.csv"


# Cohort1 = True
# Offboarding = False


In [73]:
# if Cohort1 == True:
#     subjects = pd.read_csv(c1_subjectsPath)
#     subjects = subjects.rename(columns={'Student Number': 'student_id'})
    
#     if Offboarding == True:
#         qualtrics_path = c1_qualtricsOffboardingPath
#         expfactoryPath = c1_expOffboardingPath
#         outputDirectory = c1_outputDirectory + "/battery_cohort1_offB.csv"
#     else:
#         qualtrics_path = c1_qualtricsOnboardingPath
#         expfactoryPath = c1_expOnboardingPath
#         outputDirectory = c1_outputDirectory + "/battery_cohort1_onB.csv"
# else:
    
    
#     if Offboarding == True:
#         qualtrics_path = c2_qualtricsOffboardingPath
#         expfactoryPath = c2_expOffboardingPath
#         outputDirectory = c2_outputDirectory + "/battery_cohort2_offB.csv"
#     else:
#         qualtrics_path = c2_qualtricsOnboardingPath
#         expfactoryPath = c2_expOnboardingPath
#         outputDirectory = c2_outputDirectory + "/battery_cohort2_onB.csv"

## Qualtrics

In [74]:
# Calculate qualtrics survey scores

def calc_survey_score(data: dict, startswith: bool, finder, name, mean_axis: int, likert: int, reverse='') -> None:
    cols = []
    if startswith == True:
        cols = [col for col in data if col.startswith(finder)]
    else:
        cols = [col for col in data if col.endswith(finder)]
    
    temp = data[cols].apply(pd.to_numeric)
    
    ## Control for reverse scoring
    if reverse != '':
        cols_r = [col for col in temp if (col.endswith(reverse))]
        temp[cols_r] = likert + 1 - temp[cols_r]
    
    data[name] = temp.mean(axis=mean_axis)
    return

In [75]:
## Qualtrics preprocessing
df = pd.read_csv('/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/run_1/run1_qualtrics_onboardingOffboarding.csv')
## remove first two rows
df_clean = df.loc[2:,].reset_index(drop=True)
pd.get_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [81]:
## Flourishing Scale
calc_survey_score(df_clean, True, 'flourishing', 'flourishing_score', 1, 7)

## Rapid Food Screener
calc_survey_score(df_clean, True, 'food_fruit', 'food_screener_score_fruit_veg', 1, 6)
calc_survey_score(df_clean, True, 'food_fat', 'food_screener_score_fat', 1, 5)

## Harmony of Life
calc_survey_score(df_clean, True, 'harmony_', 'harmony_score', 1, 7)

## Quality of Life
calc_survey_score(df_clean, True, 'qol', 'quality_of_life_score', 1, 7)

## Satisfaction with life Scale (SWLS)
calc_survey_score(df_clean, True, 'swl', 'SWLS_score', 1, 7)

## Subjective Happiness Scale
calc_survey_score(df_clean, True, 'happiness', 'subjective_happiness_score', 1, 7, reverse='r')

# ## Social Media Use Questionnaire
# calc_survey_score(df_clean, True, 'smuq', 'social_media_use_score', 1, 4)

## The Marlowe-Crowne Social Desirability Scale
calc_survey_score(df_clean, True, 'social_des', 'social_desirability_score', 1, 2, 'r')

## The HEXACO–60: A Short Measure of the Major Dimensions of Personality | Conscientiousness
## 6 point Likert
## Getting Hexaco columns and correcting for reverse scoring
cols = [col for col in df_clean if col.startswith('con_hex')]
temp = df_clean[cols].apply(pd.to_numeric)
cols_r = [col for col in temp if (col.endswith('r'))]
temp[cols_r] = 6 - temp[cols_r]
    
## Getting appropriate subscale columns
org = [col for col in temp if (col.startswith('con_hex_1') or col.startswith('con_hex_5'))]
dil = [col for col in temp if (col.startswith('con_hex_2') or col.startswith('con_hex_6'))]
per = [col for col in temp if (col.startswith('con_hex_3') or col.startswith('con_hex_7') or col.startswith('con_hex_9'))] 
pru = [col for col in temp if (col.startswith('con_hex_4') or col.startswith('con_hex_8') or col.startswith('con_hex_10'))] 

df_clean['con_hex_org_score'] = temp[org].mean(axis=1)
df_clean['con_hex_dil_score'] = temp[dil].mean(axis=1)
df_clean['con_hex_perf_score'] = temp[per].mean(axis=1)
df_clean['con_hex_prudence_score'] = temp[pru].mean(axis=1)

In [82]:
## Overall Qualtrics Results
cols = [col for col in df_clean if (col.endswith('score') or col.startswith('consent_sonaID'))]
qualtrics_results = df_clean[cols]

In [83]:
qualtrics_results.head()

,consent_sonaID,sat_score,lsat_score,flourishing_score,harmony_score,quality_of_life_score,SWLS_score,subjective_happiness_score,social_desirability_score,con_hex_org_score,con_hex_dil_score,con_hex_perf_score,con_hex_prudence_score
1,1.009418e+09,NaN,NaN,5.750,4.6,4.4375,4.6,4.25,1.538462,3.666667,3.0,2.666667,3.333333
3,1.008888e+09,NaN,NaN,5.625,5.6,4.4375,4.2,4.50,1.384615,3.666667,4.5,4.000000,3.666667
4,1.009766e+09,NaN,NaN,6.250,5.4,6.1875,4.6,4.75,1.846154,3.666667,4.0,4.000000,3.000000
5,1.007264e+09,NaN,NaN,2.250,3.8,5.3750,3.6,1.50,1.384615,3.000000,5.0,4.666667,1.000000
6,1.007189e+09,NaN,NaN,5.375,4.2,4.6250,2.6,3.50,1.461538,4.666667,4.0,4.000000,4.333333


## Experiment Factory

In [177]:
directory = expfactoryPath + "/"

for participantFolder in os.listdir(expfactoryPath):
    if participantFolder.startswith('.'):
        continue
    
    path = directory + participantFolder

    ## Name and ID
    name = np.nan
    student_id = ""
    try:
        name, student_id = get_id(path + '/sona-id-entry-results.json')
    except FileNotFoundError:
        pass

In [184]:
## Iterating through directories to analyze all participants
## Merging the Exp_Factory and Qualtrics Results into one DataFrame
from json.decoder import JSONDecodeError
import warnings
warnings.filterwarnings('ignore')

# qualtrics_fields = ['sat_score', 'lsat_score', 'flourishing_score', 'harmony_score', 'quality_of_life_score',
#                     'SWLS_score', 'SWL_Cathril_ladder_score', 'self_esteem_single_score', 'subjective_happiness_score',
#                     'con_hex_org_score', 'con_hex_dil_score', 'con_hex_perf_score', 'con_hex_prudence_score',
#                     'social_media_use_score', 'social_desirability_score']

df_list = []
directory = expfactoryPath + "/"

for participantFolder in os.listdir(expfactoryPath):
    if participantFolder.startswith('.'):
        continue
    
    path = directory + participantFolder

    ## Name and ID
    name = ""
    student_id = ""
    try:
        name, student_id = get_id(path + '/sona-id-entry-results.json')
    except FileNotFoundError:
        pass
    
    
    ## Attention Network Task (Eisenberg code)
    attention_network_acc = np.nan
    attention_network_congruent_rt = np.nan
    attention_network_incongruent_rt = np.nan
    attention_network_missed_percent = np.nan
    attention_network_conflict_rt = np.nan
    attention_network_conflict_acc = np.nan
    attention_network_alerting_acc = np.nan
    attention_network_alerting_rt = np.nan
    attention_network_orienting_rt = np.nan
    attention_network_orienting_acc = np.nan
    attention_network_neutral_rt  = np.nan
    attention_network_post_error_slowing = np.nan
    try:
        ATN_DV = ATN(path + '/attention-network-task-results.json').calc_attention_network_task_DV()
        attention_network_acc = ATN_DV['acc']['value']
        attention_network_missed_percent =  ATN_DV['missed_percent']['value']
        attention_network_congruent_rt = ATN_DV['congruent_rt']['value']
        attention_network_incongruent_rt = ATN_DV['incongruent_rt']['value']
        attention_network_conflict_rt = ATN_DV['conflict_rt']['value']
        attention_network_conflict_acc = ATN_DV['conflict_acc']['value']
        attention_network_alerting_acc = ATN_DV['alerting_acc']['value']
        attention_network_alerting_rt = ATN_DV['alerting_rt']['value']
        attention_network_orienting_rt = ATN_DV['orienting_rt']['value']
        attention_network_orienting_acc = ATN_DV['orienting_acc']['value']
        attention_network_neutral_rt  = ATN_DV['neutral_rt']['value']
        attention_network_post_error_slowing = ATN_DV['post_error_slowing']['value']
    except FileNotFoundError:
        pass


    ## Berlin Numeracy
    berlin_numeracy_score = np.nan
    try:
        berlin_numeracy_score = get_berlin_numeracy_score(path + '/berlin-numeracy-test-results.json')
    except FileNotFoundError:
        pass
    except JSONDecodeError:
        berlin_numeracy_score = "wrongFileFormat"
    except KeyError:
        berlin_numeracy_score = "keyError"


    ## Bickel Titrator
    bickel_titrator_score = np.nan


    ## Boredom Proneness Scale
    boredom_proneness_score = np.nan
    try:
        boredom_proneness_score = get_boredom_proneness_dvs(path + '/boredom-proneness-scale-short-survey-results.json')
    except FileNotFoundError:
        pass


    ## Brief Self Control Survey
    bscs_score = np.nan
    try:
        bscs_score = get_bscs_dvs(path +  '/brief-self-control-survey-results.json')
    except FileNotFoundError:
        pass

    
    ## Brief Sensation Seeking Scale
        BSSS_score = np.nan
        bsss_subscales = np.nan
        bsss_exp_seek = np.nan
        bsss_boredom_sus = np.nan
        bsss_thrill = np.nan
        bsss_disinhibition = np.nan
    try:
        BSSS_score, bsss_subscales = get_BSSS_score(path +  '/brief-sensation-seeking-survey-results.json')
        bsss_exp_seek = bsss_subscales['exp_seek']
        bsss_boredom_sus = bsss_subscales['boredom_sus']
        bsss_thrill = bsss_subscales['thrill']
        bsss_disinhibition = bsss_subscales['disinhibition']
    except FileNotFoundError:
        pass

   
    ## cognitive_estimation_test
    cognitive_estimation_score = np.nan
    try:
        cognitive_estimation_score = get_cognitive_estimation_test_scores(path + '/cognitive-estimation-test-results.json')
    except FileNotFoundError:
        pass
    except KeyError:
        pass

    
    ## Cognitive Reflection Task
    cognitive_ref_correct_prop = np.nan
    cognitive_ref_intuitive_prop  = np.nan
    try:
        cognitive_ref_correct_prop, cognitive_ref_intuitive_prop = get_cognitive_ref_dv(path +  '/cognitive-reflection-survey-results.json')
    except FileNotFoundError:
        pass

    ## Competetiveness Index
    competetiveness_index_enjoyment_overall_score = np.nan
    competetiveness_index_enjoyment_score = np.nan
    competetiveness_index_contentiousness_score = np.nan
    try:
        competetiveness_index_enjoyment_overall_score, competetiveness_index_enjoyment_score, competetiveness_index_contentiousness_score = get_competetiveness_index_score(path +  '/competitiveness-index-revised-survey-results.json')
    except FileNotFoundError:
        pass

    
    ## DASS 21
    DASS_overall = np.nan
    DASS_depression = np.nan
    DASS_anxiety = np.nan
    DASS_stress = np.nan
    try:
        DASS_overall, DASS_depression, DASS_anxiety, DASS_stress = get_DASS_scores(path +  '/dass-21-results.json')
    except FileNotFoundError:
        pass


    ## Effort Avoidance Task
    effort_avoidance_score = np.nan
    try:
        effort_avoidance_score = get_effort_avoidance_threshold(path + '/effort-avoidance-task-results.json')
    except FileNotFoundError:
        pass

    ## Emotion Regulation Task
    emotion_regulation_score = np.nan


    ## Five Item Ambition Scale
    ambition_score = np.nan
    try:
        ambition_score = get_ambition_score(path +  '/five-item-ambition-survey-results.json')
    except FileNotFoundError:
        pass

        
    ## Future Time Perspective
    future_time_perspective_score = np.nan
    try:
        future_time_perspective_score = get_future_time_perspective_scores(path + '/future-time-perspective-survey-results.json')
    except FileNotFoundError:
        pass


    ## General Self Efficacy Scale
    general_self_efficacy_score = np.nan
    try:
        eneral_self_efficacy_score = get_gses(path +  '/general-self-efficacy-survey-results.json')
    except FileNotFoundError:
        pass


    ## Go NoGO Task (Eisenberg code)
    go_nogo_dvs = np.nan
    go_nogo_dprime = np.nan
    go_nogo_bias = np.nan
    try:
        go_nogo_dvs = get_go_nogo_dvs(path +  '/go-nogo-results.json')
        go_nogo_dprime = go_nogo_dvs['dprime']['value']
        go_nogo_bias = go_nogo_dvs['bias']['value']
    except FileNotFoundError:
        pass

    
    ## Grit Scale
    grit_scale_score = np.nan
    grit_scale_PE = np.nan
    grit_scale_CI = np.nan
    try:
        grit_scale_score, grit_scale_PE, grit_scale_CI = get_grit_scale_score(path +  '/grit-scale-survey-results.json')
    except FileNotFoundError:
        pass

    
    ## Holt Laury Task
    holt_laury_score = np.nan
    try:
        holt_laury_score = get_risk_aversion_score(path +  '/holt-laury-survey-results.json')
    except FileNotFoundError:
        pass

    ## Information Sampling Task
    information_sampling_score = np.nan


    ## JDM Classics
    jdm_classics_score = np.nan


    ## K6 Survey
    k6_survey_score = np.nan


    ## Keep Track Score (Eisenberg code)
    keep_track_score = np.nan
    try:
        keep_track_score = KeepTrackTask(path + '/keep-track-results.json').calc_keep_track_DV()
    except FileNotFoundError:
        pass
    except KeyError:
        pass


    ## Leisure time activity
    leisure_time_activity_score = np.nan
    try:
        leisure_time_activity_score = get_leisure_time_activity_results(path + '/leisure-time-activity-survey-results.json')
    except FileNotFoundError:
        pass

    
    ## Life Orientation Task
    life_orientation_test_score = np.nan
    try:
        life_orientation_test_score = get_lot_score(path +  '/life-orientation-test-revised-optimism-survey-results.json')
    except FileNotFoundError:
        pass


    ## Maximizing Tendency Scale
    ms_dict = np.nan
    maximizing_scale_overall_score =  np.nan
    maximizing_scale_alt_search = np.nan
    maximizing_scale_dec_difficulty = np.nan
    maximizing_scale_high_standards = np.nan
    maximizing_scale_MTS = np.nan
    maximizing_scale_SMTS = np.nan
    try:
        ms_dict, maximizing_scale_overall_score =  get_maximizing_scale_results(path +  '/maximizing-tendency-survey-results.json')
        maximizing_scale_alt_search = ms_dict['alternative_search']
        maximizing_scale_dec_difficulty = ms_dict['decision_difficulty']
        maximizing_scale_high_standards = ms_dict['high_standards']
        maximizing_scale_MTS = ms_dict['MTS']
        maximizing_scale_SMTS = ms_dict['SMTS']
    except FileNotFoundError:
        pass


    ## Mindful Attention Awareness Survey
    mindful_attention = np.nan
    try:
        mindful_attention = get_mindfulness_attention_score(path + '/mindful-attention-awareness-survey-results.json')
    except FileNotFoundError:
        pass


    ## Need for cognition Survey
    need_for_cognition_score = np.nan
    try:
        need_for_cognition_score = get_need_for_cognition_score(path+  '/need-for-cognition-and-faith-in-intuition-survey-results.json')
    except FileNotFoundError:
        pass


    ## Perceived Stress Scale
    perceived_stress_score = np.nan
    try:
        perceived_stress_score = get_pss_score(path +  '/perceived-stress-survey-results.json')
    except FileNotFoundError:
        pass
    

    ## Plus Minus Task
    plus_minus_score = np.nan
    try:
        plus_minus_score = get_plus_minus_dv(path +  '/plus-minus-results.json')
    except FileNotFoundError:
        pass
    except KeyError:
        pass


    ## PANAS Scale
    positive_and_negative_subscales = np.nan
    PANAS_positive_affect = np.nan
    PANAS_negative_affect = np.nan
    try:
        positive_and_negative_subscales = get_positive_negative_affect_score(path +  '/positive-and-negative-affect-short-survey-results.json')
        PANAS_positive_affect = positive_and_negative_subscales['positive_affect']
        PANAS_negative_affect = positive_and_negative_subscales['negative_affect']
    except FileNotFoundError:
        pass


    ## Probabilistic Selection Task
    probabilistic_selc_dvs = np.nan
    value_sensitivity = np.nan
    positive_learning_bias = np.nan
    # try:
    #     probabilistic_selc_dvs = get_probabilistic_selc_dv(path + '/probabilistic-selection-results.json')
    #     value_sensitivity = probabilistic_selc_dvs['positive_learning_bias']['value']
    #     positive_learning_bias = probabilistic_selc_dvs['value_sensitivity']['value']
    # except FileNotFoundError:
    #     pass


    ## Propensity To Plan Task
    propensity_to_plan_msr = np.nan
    propensity_to_plan_mlr = np.nan
    propensity_to_plan_tsr = np.nan
    propensity_to_plan_tlr = np.nan
    try:
        propensity_to_plan_msr, propensity_to_plan_mlr, propensity_to_plan_tsr, propensity_to_plan_tlr = get_propensity_to_plan_score(path +  '/propensity-to-plan-survey-results.json')
    except FileNotFoundError:
        pass


    ## Psychomotor Vigilance Task
    psychomotor_vigilance_score = np.nan
    try:
        psychomotor_vigilance_score = get_psychomotor_vigilance(path + '/psychomotor-vigilance-test-results.json')
    except FileNotFoundError:
        pass


    ## RAVENS Task (Eisenberg code)
    ravens_score = np.nan
    try:
        ravens_dvs = Ravens(path + '/ravens-results.json').calc_ravens_DV()
        ravens_score = ravens_dvs['score']['value']
    except:
        pass
    

    ## Rosenberg Self Esteem Survey
    rosenberg_SES_score = np.nan
    try:
        rosenberg_SES_score = get_rosenberg_SES_score(path +  '/rosenberg-self-esteem-survey-results.json')
    except FileNotFoundError:
        pass

    
    ## Secular Measure of Work Ethic
    smwe_dict = np.nan
    secular_measure_work_ethic_overall_score = np.nan
    secular_measure_work_ethic_cli_score = np.nan
    secular_measure_work_ethic_moral_score = np.nan
    secular_measure_work_ethic_iwm_score = np.nan
    try:
        smwe_dict, secular_measure_work_ethic_overall_score = get_secular_measure_work_ethic_score(path +  '/secular-measure-of-work-ethic-survey-results.json')
        secular_measure_work_ethic_cli_score = smwe_dict['cli_score']
        secular_measure_work_ethic_moral_score = smwe_dict['moral_score']
        secular_measure_work_ethic_iwm_score = smwe_dict['iwm_score']
    except FileNotFoundError:
        pass


    ## Semantic Verbal Fluency
    semantic_verbal_fluency_task = np.nan


    ## Shift Task
    shift_task_score = np.nan


    ## Simon Task (Eisenberg code)
    simon_missed_percent = np.nan
    simon_post_error_slowing = np.nan
    simon_rt = np.nan
    simon_acc = np.nan
    simon_congruent_seq_rt = np.nan
    simon_incongruent_seq_acc = np.nan
    simon_congruent_acc = np.nan
    simon_incongruent_acc = np.nan
    simon_congruent_avg_rt = np.nan
    simon_incongruent_avg_rt = np.nan
    try:
        simon_dvs = Simon(path + '/simon-results.json').calc_simon_DV()
        simon_missed_percent = simon_dvs['missed_percent']['value']
        simon_post_error_slowing = simon_dvs['post_error_slowing']['value']
        simon_rt = simon_dvs['simon_rt']['value']
        simon_acc = simon_dvs['simon_acc']['value']
        simon_congruent_seq_rt = simon_dvs['congruency_seq_rt']['value']
        simon_incongruent_seq_acc = simon_dvs['congruency_seq_acc']['value']
        simon_congruent_acc = simon_dvs['congruent_acc']['value']
        simon_incongruent_acc = simon_dvs['incongruent_acc']['value']
        simon_congruent_avg_rt = simon_dvs['congruent_avg_rt']['value']
        simon_incongruent_avg_rt = simon_dvs['incongruent_avg_rt']['value']
    except FileNotFoundError:
        pass
    except KeyError:
        pass
    
    ## Simple Reaction Time (Eisenberg code)
    simple_reaction_time_avg_rt = np.nan
    simple_reaction_time_missed_percent = np.nan
    try:
        simpleRTDV = SimpleRT(path + '/simple-reaction-time-results.json').calc_simple_RT_DV()[0]
        simple_reaction_time_avg_rt = simpleRTDV['avg_rt']['value']
        simple_reaction_time_missed_percent = simpleRTDV['missed_percent']['value']
    except FileNotFoundError:
        pass
    except KeyError:
        pass


    ## Stroop Task
    stroop_results = np.nan
    stroop_effect = np.nan
    stroop_accuracy = np.nan
    stroop_con_avg = np.nan
    stroop_con_sd = np.nan
    stroop_incon_avg = np.nan
    stroop_incon_sd = np.nan
    try:
        stroop_results = get_stroop_dvs(path +  '/stroop-results.json')
        stroop_effect = stroop_results['stroop_effect']
        stroop_accuracy = stroop_results['stroop_accuracy']
        stroop_con_avg = stroop_results['stroop_con_avg']
        stroop_con_sd = stroop_results['stroop_con_sd']
        stroop_incon_avg = stroop_results['stroop_incon_avg']
        stroop_incon_sd = stroop_results['stroop_incon_sd']
    except FileNotFoundError:
        pass


    ## Tower of London Task (Eisenberg code)
    TOL_num_correct = np.nan
    TOL_num_optimal_solutions = np.nan
    TOL_num_extra_moves = np.nan
    TOL_planning_time = np.nan
    TOL_avg_move_time = np.nan
    TOl_weighted_performance_score = np.nan
    try:
        TOL_DV = TOL(path + '/tower-of-london-results.json').calc_TOL_DV()[0]
        TOL_num_correct = TOL_DV['num_correct']['value']
        TOL_num_optimal_solutions = TOL_DV['num_optimal_solutions']['value']
        TOL_num_extra_moves = TOL_DV['num_extra_moves']['value']
        TOL_planning_time = TOL_DV['planning_time']['value']
        TOL_avg_move_time = TOL_DV['avg_move_time']['value']
        TOl_weighted_performance_score = TOL_DV['weighted_performance_score']['value']
    except FileNotFoundError:
        pass
    except KeyError:
        pass
    except TypeError:
        pass



    ## Trait Emotional Intelligence
    trait_emotional_intelligence_score = np.nan
    try:
        trait_emotional_intelligence_score = get_TEI_score(path +  '/trait-emotional-intelligence-short-survey-results.json')
    except FileNotFoundError:
        pass


    ## Trait Hedonic Capacity
    trait_hedonic_capacity_score = np.nan
    try:
        trait_hedonic_capacity_score = get_THC_score(path +  '/trait-hedonic-capacity-survey-results.json')
    except FileNotFoundError:
        pass


    ## Trait Robustness of Self Confidence
    trait_robustness_of_self_confidence_score = np.nan
    try:
        trait_robustness_of_self_confidence_score = get_trsc_score(path +  '/trait-robustness-of-self-confidence-survey-results.json')
    except FileNotFoundError:
        pass


    ## Two Stage Decision Results
    two_stage_decision_score = np.nan


    data = {'name': name, 'student_id': student_id, 
            
            'attention_network_acc': attention_network_acc,
            'attention_network_congruent_rt': attention_network_congruent_rt,
            'attention_network_incongruent_rt': attention_network_incongruent_rt,
            'attention_network_missed_percent': attention_network_missed_percent,
            'attention_network_conflict_rt': attention_network_conflict_rt,
            'attention_network_conflict_acc': attention_network_conflict_acc,
            'attention_network_alerting_acc': attention_network_alerting_acc,
            'attention_network_alerting_rt': attention_network_alerting_rt,
            'attention_network_orienting_rt': attention_network_orienting_rt,
            'attention_network_orienting_acc': attention_network_orienting_acc,
            'attention_network_neutral_rt': attention_network_neutral_rt,
            'attention_network_post_error_slowing': attention_network_post_error_slowing,

            'berlin_numeracy_score': berlin_numeracy_score, 'bickel_titrator_score': bickel_titrator_score,
            'boredom_proneness_score': boredom_proneness_score, 'brief_self_control_score': bscs_score, 
            'bsss_overall': BSSS_score, 'bsss_exp_seek': bsss_exp_seek,
            'bsss_boredom_sus': bsss_boredom_sus, 'bsss_thrill': bsss_thrill, 'bsss_disinhibition': bsss_disinhibition, 
            
            'cognitive_estimation_score': cognitive_estimation_score, 'cognitive_ref_correct_prop': cognitive_ref_correct_prop, 
            'cognitive_ref_intuitive_prop': cognitive_ref_intuitive_prop, 'competetiveness_index_enjoyment_overall': competetiveness_index_enjoyment_overall_score,
            'competetiveness_index_enjoyment': competetiveness_index_enjoyment_score, 'competetiveness_index_contentiousness': competetiveness_index_contentiousness_score,
            'DASS_overall': DASS_overall, 'DASS_depression': DASS_depression, 'DASS_anxiety': DASS_anxiety, 'DASS_stress': DASS_stress, 

            'effort_avoidance_score': effort_avoidance_score, 'emotion_regulation_score': emotion_regulation_score, 'ambition_score': ambition_score,
            'future_time_perspective_score': future_time_perspective_score, 'general_self_efficacy_score': general_self_efficacy_score, 'go_nogo_dprime': go_nogo_dprime,
            'go_nogo_bias': go_nogo_bias, 'grit_scale_score': grit_scale_score, 'grit_scale_PE': grit_scale_PE, 'grit_scale_CI': grit_scale_CI, 
            
            'holt_laury_score': holt_laury_score, 'information_sampling_score': information_sampling_score, 'jdm_classics_score': jdm_classics_score, 'k6_survey_score': k6_survey_score,
            'keep_track_score': keep_track_score, 'leisure_time_activity_score': leisure_time_activity_score, 'life_orientation_test_score': life_orientation_test_score, 

            'maximizing_scale_overall': maximizing_scale_overall_score, 'maximizing_scale_alt_search': maximizing_scale_alt_search, 'maximizing_scale_dec_difficulty': maximizing_scale_dec_difficulty, 
            'maximizing_scale_high_standards': maximizing_scale_high_standards, 'maximizing_scale_MTS': maximizing_scale_MTS, 'maximizing_scale_SMTS': maximizing_scale_SMTS,
            'mindful_attention': mindful_attention, 'need_for_cognition_score': need_for_cognition_score, 
            
            'perceived_stress_score': perceived_stress_score, 'plus_minus_score': plus_minus_score, 
            'PANAS_positive_affect': PANAS_positive_affect, 'PANAS_negative_affect': PANAS_negative_affect,  'value_sensitivity': value_sensitivity, 'positive_learning_bias': positive_learning_bias,
            'propensity_to_plan_msr': propensity_to_plan_msr, 'propensity_to_plan_mlr': propensity_to_plan_mlr, 'propensity_to_plan_tsr':propensity_to_plan_tsr, 
            'propensity_to_plan_tlr': propensity_to_plan_tlr, 'psychomotor_vigilance_score': psychomotor_vigilance_score,
            
            'ravens_score': ravens_score, 'rosenberg_SES_score': rosenberg_SES_score, 
            'secular_measure_work_ethic_score': secular_measure_work_ethic_overall_score, 'secular_measure_work_ethic_cli': secular_measure_work_ethic_cli_score, 
            'secular_measure_work_ethic_moral': secular_measure_work_ethic_moral_score, 'secular_measure_work_ethic_iwm': secular_measure_work_ethic_iwm_score, 
            'semantic_verbal_fluency_task': semantic_verbal_fluency_task, 'shift_task_score': shift_task_score,

            'simon_missed_percent': simon_missed_percent, 'simon_post_error_slowing': simon_post_error_slowing, 'simon_rt': simon_rt, 'simon_acc': simon_acc,
            'simon_congruent_seq_rt': simon_congruent_seq_rt, 'simon_incongruent_seq_acc': simon_incongruent_seq_acc, 'simon_congruent_acc': simon_congruent_acc, 
            'simon_incongruent_acc': simon_incongruent_acc, 'simon_congruent_avg_rt': simon_congruent_avg_rt, 'simon_incongruent_avg_rt': simon_incongruent_avg_rt,
            'simple_reaction_time_avg_rt': simple_reaction_time_avg_rt, 'simple_reaction_time_missed_percent': simple_reaction_time_missed_percent,
            'stroop_effect': stroop_effect, 'stroop_accuracy': stroop_accuracy, 'stroop_con_avg': stroop_con_avg, 'stroop_con_sd': stroop_con_sd, 'stroop_incon_avg': stroop_incon_avg, 
            'stroop_incon_sd': stroop_incon_sd, 'TOL_num_correct': TOL_num_correct, 'TOL_num_optimal_solutions': TOL_num_optimal_solutions,
            'TOL_num_extra_moves': TOL_num_extra_moves, 'TOL_planning_time': TOL_planning_time, 'TOL_avg_move_time': TOL_avg_move_time, 'TOl_weighted_performance_score': TOl_weighted_performance_score,
            
            'trait_emotional_intelligence_score': trait_emotional_intelligence_score, 'trait_hedonic_capacity_score': trait_hedonic_capacity_score, 
            'trait_robustness_of_self_confidence_score': trait_robustness_of_self_confidence_score, 'two_stage_decision_score': two_stage_decision_score}
    

    
    # ## Merging with Qualtrics Results
    # determinant = 100
    # temp = []
    # if name in qualtrics_results.values:
    #     temp = qualtrics_results.loc[qualtrics_results['consent_sonaID'] == name]
    # elif student_id in qualtrics_results.values:
    #     temp = qualtrics_results.loc[qualtrics_results['consent_sonaID'] == student_id]
    # else:
    #     determinant = 0
    
    # for item in qualtrics_fields:
    #     if determinant != 0:
    #         temp = temp.reset_index(drop=True)
    #         data[item] = temp.at[0, item]
    #     else:
    #         data[item] = np.nan
    
    ## Appending the participant into the future dataframe list
    df_list.append(data)

In [185]:
## Final results
results = pd.DataFrame(df_list)
## results.to_csv(outputDirectory, index=False)

In [186]:
results

,name,student_id,attention_network_acc,attention_network_congruent_rt,attention_network_incongruent_rt,attention_network_missed_percent,attention_network_conflict_rt,attention_network_conflict_acc,attention_network_alerting_acc,attention_network_alerting_rt,...,TOL_num_correct,TOL_num_optimal_solutions,TOL_num_extra_moves,TOL_planning_time,TOL_avg_move_time,TOl_weighted_performance_score,trait_emotional_intelligence_score,trait_hedonic_capacity_score,trait_robustness_of_self_confidence_score,two_stage_decision_score
0,hussa983,1009008574,0.960265,690.0,899.0,0.811543,209.0,-0.120000,-0.075000,64.0,...,9.0,7.0,12.0,1876.083333,2219.018254,28.0,4.300000,4.3,5.125,NaN
1,yosofiha,1009159198,0.666667,666.5,768.5,0.897527,102.0,-0.057971,0.022222,278.0,...,10.0,7.0,17.0,3697.583333,1787.685238,26.0,4.100000,3.2,4.250,NaN
2,zhan9168,1006740270,1.000000,510.0,563.5,0.793875,53.5,0.000000,0.000000,41.5,...,11.0,9.0,8.0,4503.454545,1371.722222,24.0,3.866667,2.9,4.500,NaN
3,xuziya13,1008968663,0.969512,631.0,712.0,0.797409,81.0,-0.057615,-0.001161,171.5,...,11.0,11.0,1.0,7268.363636,1617.333333,36.0,3.433333,2.9,4.125,NaN
4,mathure1,1008882032,0.928144,511.0,534.0,0.792697,23.0,-0.127597,0.046458,72.0,...,12.0,9.0,21.0,2692.083333,2015.759396,42.0,4.200000,4.5,4.625,NaN
5,guerre70,1007022687,0.986577,580.0,842.0,0.813899,262.0,-0.020408,-0.000675,-41.0,...,12.0,10.0,4.0,4050.416667,2310.041667,37.0,4.200000,3.3,4.125,NaN
6,davidsus,1009281199,0.742331,683.0,828.0,0.797409,145.0,-0.807692,-0.031098,-39.0,...,9.0,4.0,19.0,3811.750000,2646.780423,15.0,4.233333,3.4,4.375,NaN
7,likare22,1008915952,0.948905,676.0,891.5,0.830389,215.5,-0.045290,0.089076,31.0,...,12.0,10.0,7.0,4214.750000,1578.524242,34.0,3.433333,3.8,3.500,NaN
8,mikatino,1009251015,0.994048,554.5,638.0,0.791519,83.5,-0.017857,0.000000,-14.5,...,12.0,6.0,16.0,3187.000000,1838.730303,28.0,4.266667,3.1,4.000,NaN
9,lawalfaa,1009264136,0.950920,581.0,614.5,0.797409,33.5,-0.036338,-0.023780,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.600000,3.4,4.375,NaN


In [187]:
# filter for subjects in cohort
subjects = subjects.dropna(subset=['student_id']) # can't match without student id
subjects.student_id = subjects.student_id.astype(int).astype(str) # convert so matches type of results student_id
results = results[results['student_id'].isin(subjects.student_id)]
# add participant identifier
results = pd.merge(results, subjects[['student_id', 'ParticipantIdentifier']], on='student_id', how='left')

# drop name and student ID column for privacy
results = results.drop(['name'], axis=1)

In [189]:
# Save to csv
results.to_csv(outputDirectory, index=False)

### Select from subjects

In [148]:
subs = pd.read_csv(subjectsPath)

In [149]:
subs = subs[['ParticipantIdentifier', 'Student Number']]

In [150]:
subs.columns = ['ParticipantIdentifier', 'student_id']

In [151]:
subs = subs.dropna().reset_index(drop=True)

In [152]:
subs.student_id = subs.student_id.astype('int32')
subs.student_id = subs.student_id.astype('str')

#### Merge with onboarding

In [153]:
results = subs.merge(results, how='left', on='student_id')

In [154]:
results.drop(['name', 'student_id'], axis=1, inplace=True)

In [155]:
results = results[results['attention_network_acc'].notna()].reset_index(drop=True)

In [156]:
cols = ['onB_' + str(col) for col in results.columns]

In [157]:
cols[0] = 'ParticipantIdentifier'

In [160]:
results.columns = cols

In [162]:
battery = results

#### Merge with offboarding

In [189]:
results = subs.merge(results, how='left', on='student_id')

In [190]:
results.drop(['name', 'student_id'], axis=1, inplace=True)

In [191]:
results = results[results['attention_network_acc'].notna()].reset_index(drop=True)

In [192]:
cols = ['offB_' + str(col) for col in results.columns]

In [193]:
cols[0] = 'ParticipantIdentifier'

In [194]:
results.columns = cols

In [204]:
battery.to_csv(outputDirectory + 'run1_battery.csv', index=False)